In [1]:
import sys
sys.path.append("../")

In [2]:
import pandas as pd
df = pd.read_csv(
    "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/players_raw.csv"
)
df2 = pd.read_csv(
    "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/f0cd9a537e4daff00a9a8b0d8019bc4e28da94d3/data/2019-20/players_raw.csv"
)
df = pd.merge(df[["first_name", "second_name", "total_points", "team_code", "element_type", "now_cost", "id"]],
              df2[["id", "total_points"]],
              on="id", suffixes=["", "_last_season"])
df.head()

old_expected_scores = df["total_points_last_season"] / 38
new_expected_scores = (0.9*old_expected_scores + df["total_points"]*0.1).fillna(0)  # weighted average points from last season and points last week
prices = df["now_cost"] / 10
positions = df["element_type"]
clubs = df["team_code"]
# so we can read the results
names = df["first_name"] + " " + df["second_name"]
# retrieve the team I picked last week

In [3]:
from fpl_opt.models import TransferOptimiser, select_team

In [4]:
opt = TransferOptimiser(new_expected_scores, prices, prices, positions, clubs)

In [5]:
decisions, captain_decisions, sub_decisions = select_team(old_expected_scores.values, prices.values, positions.values, clubs.values)
player_indices = []
for i in range(len(decisions)):
    if decisions[i].value() == 1:
        print(names[i])
        player_indices.append(i)

sub_indices = []
for i in range(len(sub_decisions)):
    if sub_decisions[i].value() == 1:
        print(names[i])
        sub_indices.append(i)

captain_indices = []
for i in range(len(captain_decisions)):
    if captain_decisions[i].value() == 1:
        print(names[i])
        captain_indices.append(i)

player_indices, sub_indices, captain_indices

Total expected score = 62.73684210526316
David Luiz Moreira Marinho
Ryan Fraser
Luka Milivojevic
Lucas Digne
Gylfi Sigurdsson
Andrew Robertson
Virgil van Dijk
Mohamed Salah
Aymeric Laporte
Ederson Santana de Moraes
Raúl Jiménez
Edward Nketiah
N'Golo Kanté
Shane Long
Lukasz Fabianski
Mohamed Salah


([20, 72, 171, 183, 193, 234, 236, 244, 256, 265, 508],
 [12, 150, 398, 484],
 [244])

In [6]:
opt.solve(player_indices, sub_indices, captain_indices, budget_now=0, sub_factor=0.1)

KeyError: 'min_starters'